In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pyfume.Clustering import Clusterer
from pyfume.EstimateAntecendentSet import AntecedentEstimator
from pyfume.EstimateConsequentParameters import ConsequentEstimator
from pyfume.SimpfulModelBuilder import SugenoFISBuilder
from pyfume.Tester import SugenoFISTester
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, cohen_kappa_score
from numpy import clip, column_stack, argmax


In [7]:
# %% Load dataset and create train-test sets
Train = pd.read_csv('dataTrain.csv')
Test = pd.read_csv('dataTest.csv')

In [8]:
# Prepare the data
X_train = Train.drop('STATUS', axis=1).to_numpy()
y_train = Train['STATUS'].to_numpy()
X_test = Test.drop('STATUS', axis=1).to_numpy()
y_test = Test['STATUS'].to_numpy()

var_names = Train.columns[:-1].to_list()

In [9]:
# %% Function to train Takagi-Sugeno Model
def train_takagi_sugeno(X_train, y_train, X_test, y_test, num_clusters=10, method='fcm'):
   
    # Cluster the input-output space
    cl = Clusterer(x_train=X_train, y_train=y_train, nr_clus=num_clusters)
    clust_centers, part_matrix, _ = cl.cluster(method=method)

    # Estimate membership functions parameters
    ae = AntecedentEstimator(X_train, part_matrix)
    antecedent_params = ae.determineMF()

    # Estimate consequent parameters
    ce = ConsequentEstimator(X_train, y_train, part_matrix)
    conseq_params = ce.suglms()

    # Build the Sugeno FIS model
    modbuilder = SugenoFISBuilder(antecedent_params, conseq_params, var_names, save_simpful_code=False)
    model = modbuilder.get_model()

    # Get model predictions
    modtester = SugenoFISTester(model, X_test, var_names)
    y_pred_probs = clip(modtester.predict()[0], 0, 1)
    y_pred_probs = column_stack((1 - y_pred_probs, y_pred_probs))
    y_pred = argmax(y_pred_probs, axis=1)

    # Compute classification metrics
    metrics = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred),
        "Kappa": cohen_kappa_score(y_test, y_pred),
    }

    return metrics

In [10]:
methods = ['fcm']  
results = {}

for method in methods:
    print(f"Training with method: {method}")
    try:
        metrics = train_takagi_sugeno(X_train, y_train, X_test, y_test, num_clusters=10, method=method)
        results[method] = metrics
        for metric, value in metrics.items():
            print(f"{metric}: {value:.3f}")
    except Exception as e:
        print(f"Failed with method '{method}': {e}")
    print("-" * 40)

print("Summary of Results:")
for method, metrics in results.items():
    print(f"Method: {method}")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.3f}")
    print("-" * 40)


Training with method: fcm
 * Detected 10 rules / clusters
 * Detected Sugeno model type
Accuracy: 0.876
Recall: 0.846
Precision: 0.905
F1-Score: 0.874
Kappa: 0.753
----------------------------------------
Summary of Results:
Method: fcm
  Accuracy: 0.876
  Recall: 0.846
  Precision: 0.905
  F1-Score: 0.874
  Kappa: 0.753
----------------------------------------
